In [ ]:
# 12장 동시에 실행하기
# 동시 실행과 관련된 파이썬 모듈을 사용하면 한꺼번에 여러 가지 일을 수행할 수 있다. 이번 장에서는 스레드, 멀티 프로세싱 등 동시 실행과 관련된 파이썬 모듈을 알아본다.

## 066 스레드를 이용하여 병렬로 처리하려면? ― threading


In [ ]:
# 066 스레드를 이용하여 병렬로 처리하려면? ― threading

# threading은 스레드를 이용하여 한 프로세스에서 2가지 이상의 일을 동시에 실행할 수 있게 하는 모듈이다.

# 문제
# 다음은 페이지 번호를 입력받아 위키독스의 페이지 리소스를 wikidocs_페이지번호.html 파일로 저장하도록 만든 함수이다.

# SSL 관련 오류가 발생한다면 '030 웹 페이지를 임시로 저장하려면? - functools.lru_cache'을 참고하자.

# import urllib.request


# def get_wikidocs(page):
#     print("wikidocs page:{}".format(page))  # 페이지 호출시 출력
#     resource = 'https://wikidocs.net/{}'.format(page)
#     try:
#         with urllib.request.urlopen(resource) as s:
#             with open('wikidocs_%s.html' % page, 'wb') as f:
#                 f.write(s.read())
#     except urllib.error.HTTPError:
#         return 'Not Found'

# A 씨는 참고 자료를 만드는 데 다음처럼 10개의 위키독스 페이지가 필요하다고 한다.

# 12, 13, 14, 15, 17, 18, 20, 21, 22, 24
# 앞의 함수를 사용하여 페이지 수만큼 실행하여 원하는 결과를 얻을 수 있다. 
# 하지만, A 씨는 10개의 요청을 하나씩 보내기보다는 동시에 요청하여 더 빠른 속도로 리소스를 얻고 싶어 한다. 
# 동시에 요청할 수 있도록 하려면 함수를 어떻게 수정해야 할까?

# 참고: 090 웹 페이지를 저장하려면? - urllib


In [2]:
!pip uninstall certifi
!pip install certifi


^C


In [3]:
import urllib.request

def get_wikidocs(page):
    print('wikidocs page:{}'.format(page))
    resource = 'http://wikidocs.net/{}'.format(page)
    try:
        with urllib.request.urlopen(resource) as s:
            with open('wikikocs_%s.html' % page, 'wb') as f:
                f.write(s.read())
    except urllib.error.HTTPError:
        return 'Not Found'
get_wikidocs(1)


wikidocs page:1


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>

In [ ]:

# 풀이
# 동시에 요청하려면 threading 모듈을 사용하면 된다. 먼저 이 모듈을 사용하지 않고 10개의 페이지를 요청할 때 시간이 얼마나 걸리는지 측정해 보자.

# [파일명: threading_no_sample.py]

# import urllib.request


# def get_wikidocs(page):
#     print("wikidocs page:{}".format(page))  # 페이지 호출시 출력
#     resource = 'https://wikidocs.net/{}'.format(page)
#     try:
#         with urllib.request.urlopen(resource) as s:
#             with open('wikidocs_%s.html' % page, 'wb') as f:
#                 f.write(s.read())
#     except urllib.error.HTTPError:
#         return 'Not Found'


# import time

# start = time.time()

# pages = [12, 13, 14, 15, 17, 18, 20, 21, 22, 24]
# for page in pages:
#     get_wikidocs(page)

# end = time.time()

# print("수행시간: %f 초" % (end - start))
# 이 프로그램을 실행하면 다음과 같이 출력된다.

# wikidocs page:12
# wikidocs page:13
# wikidocs page:14
# wikidocs page:15
# wikidocs page:17
# wikidocs page:18
# wikidocs page:20
# wikidocs page:21
# wikidocs page:22
# wikidocs page:24
# 수행시간: 10.096202 초
# 필자의 PC에서는 페이지 10개의 리소스를 저장하는데 10초 정도의 시간이 걸렸다.

# 예제 실행 시간은 네트워크 환경 또는 서버의 트래픽 상태에 따라서 다를 수 있다.

# 이번에는 스레드로 병렬 처리하는 threading 모듈을 사용하여 페이지 리소스를 동시에 저장하도록 변경해 보자.

# [파일명: threading_yes_sample.py]

# import urllib.request


# def get_wikidocs(page):
#     print("wikidocs page:{}".format(page))  # 페이지 호출시 출력
#     resource = 'https://wikidocs.net/{}'.format(page)
#     try:
#         with urllib.request.urlopen(resource) as s:
#             with open('wikidocs_%s.html' % page, 'wb') as f:
#                 f.write(s.read())
#     except urllib.error.HTTPError:
#         return 'Not Found'


# import time
# import threading

# start = time.time()

# pages = [12, 13, 14, 15, 17, 18, 20, 21, 22, 24]
# threads = []
# for page in pages:
#     t = threading.Thread(target=get_wikidocs, args=(page, ))
#     t.start()
#     threads.append(t)

# for t in threads:
#     t.join()  # 스레드가 종료될 때까지 대기

# end = time.time()

# print("수행시간: %f 초" % (end - start))
# threading.Thread로 get_wikidocs() 함수를 실행하는 스레드 t를 생성한다. target은 실행할 함수이고 args는 함수에 전달할 인수이다. 이렇게 스레드를 생성한 후 t.start()를 실행하면 스레드가 독립적으로 실행된다.

# 각 스레드는 이전 스레드의 종료를 기다리지 않고 곧바로 실행된다.

# 그리고 만든 각 스레드 t는 threads라는 리스트 변수에 담았다. 이렇게 하는 이유는 생성된 스레드를 threads 변수에 담아 두어야 스레드가 종료될 때까지 기다리는 join() 함수를 for문 밖에서 차례대로 호출할 수 있기 때문이다. 만약 for문 안에서 스레드를 생성하여 실행하고 join() 함수도 곧바로 호출한다면 스레드의 이점이 사라지게 된다.

# 생성된 스레드를 모두 종료하려면 생성된 스레드마다 join() 함수를 호출해야만 한다. 만약 join() 없이 이 프로그램을 실행하면 아직 스레드가 종료되지 않은 상태에서 수행 시간이 출력되므로 정확한 수행 시간을 출력할 수 없게 된다.

# threading 모듈을 사용하도록 수정한 프로그램의 실행 결과는 다음과 같다.

# wikidocs page:12
# wikidocs page:13
# wikidocs page:14
# wikidocs page:15
# wikidocs page:17
# wikidocs page:18
# wikidocs page:20
# wikidocs page:21
# wikidocs page:22
# wikidocs page:24
# 수행시간: 2.453772 초
# 10초 정도 걸리던 작업이 2.4초가량으로 상당히 단축된 것을 확인할 수 있다(실행 환경에 따라 실행 시간은 다름).

# 참고
# threading - 스레드 기반 병렬 처리: https://docs.python.org/ko/3/library/threading.html